# モデルの評価とハイパーパラメータのチューニングのベストプラクティス
- アルゴリズムをチューニングし、モデルの性能を評価する
- モデルの性能の偏りのない推定量の算出
- 機械学習のアルゴリズムに共通する問題の診断
- 機械学習のモデルのチューニング
- さまざまな性能指標に基づく予測モデルの評価

# 6.1パイプラインによるワークフローの効率化
- scikit learnのPipelineクラスは非常に便利である
- このクラスを利用すれば任意の個数の変換ステップを含んだモデルを学習し、それをお応用して新しいデータを予測できる。

# 6.1.1 Breast Cancer Wisconcinデータセットを読み込む
- このデータセットには、悪性腫瘍細胞と良性腫瘍細胞の569のサンプルが含まれている。
- 特徴量としては、細胞核のデジタル画像から算出された30個の実数値が含まれてる。
- 腫瘍が良性か悪性かを予測するモデルの構築に利用できる。

In [4]:
#データセットを読み込む
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',header=None)

In [11]:
#特徴量をX、ターゲットをyに格納
from sklearn.preprocessing import LabelEncoder
import numpy as np
X = df.loc[:,2:].values
y = df.loc[:,1].values
#クラスラベルの文字列表現を整数に変換する
le = LabelEncoder()
y = le.fit_transform(y)

In [14]:
#エンコードの確認
print le.transform(['M','B'])
#エンコードの確認２
print le.classes_

[1 0]
['B' 'M']


In [15]:
#トレーニングデータセットとテストデータセットに分割する
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=1)

# 6.1.2 パイプラインで変換器と推定器を結合する。

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

#連結する処理としてスケーリング、主成分分析、ロジスティック回帰を指定
pipe_lr = Pipeline([('scl',StandardScaler()),('pca',PCA(n_components=2)),('clf',LogisticRegression(random_state=1))])
pipe_lr.fit(X_train,y_train)
print('Test Accuracy: %.3f') %pipe_lr.score(X_test,y_test)

Test Accuracy: 0.947


- パイプラインの中間ステップである変換器の個数には制限が無い

# 6.2 K分割交差検証を使ったモデルの性能の評価
- 機械学習モデルの構築において重要なステップの１つは、モデルにとって未知のデータを使って性能を評価することである。
- バイアスとバリアンスの適度なバランスをとるには、モデルを入念に評価する必要がある

# 6.2.1 ホールドアウト法
- ホールドアウト法はモデルの汎化性能を評価するために使用される一般的アプローチである。
- 一般に機械学習を応用するには、未知のデータに対する予測性能をさらに向上させるために、様々なパラメータ設定のチューニングや比較を行うことも重要である。このプロセスはモデル選択と呼ばれる。
- モデル選択に同じテストデータセットを繰り返し利用した場合、それはトレーニングデータセットの一部となる。このため、モデルが過学習に陥る可能性が高くなる。
- モデル選択にホールドアウト法を使用する場合、より効果的な方法は、データをトレーニングデータセット、検証データセット、テストデータセットの３つに分割することである。
- トレーニングデータセットは様々なモデルの学習に使用される。
- 検証データセットでの性能は、モデル選択に使用される。
- トレーニングステップとモデル選択ステップで未知のテストデータセットを使用することにより、利点を享受できる。
- 利点とは、新しいデータを汎化する能力を評価についてモデルを評価するときにバイアスが低くなることである。
- ホールドアウト法には問題点が１つある。元のトレーニングデータセットをトレーニングサブセットと検証サブセットにどのように分割するかによって、性能の評価に影響が及ぶことである。

# 6.2.1 k分割交差検証
- トレーニングデータセットのk個のサブセットに対してホールドアウト法をｋ回繰り返す。
- 非復元抽出を用いて、トレーニングデータセットをランダムにk個に分割する。そのうちのk-1個をモデルのトレーニングに使用し、１個をテストに使用する。この手順をk回繰り返すことで、k個のモデルと性能評価を取得する。
- 個々のサブセットに基づいてモデルの平均性能を計算する。
- ホールドアウト法と比べてトレーニングデータの再分割に敏感でない性能評価を取得できる。
- 一般にk分割交差検証はモデルのチューニングに使用される。ハイパーパラメータの最適値を見つけるために行われる。
- 満足の行くハイパーパラメータ値が見つかったら、トレーニングデータセット全体でモデルを再トレーニングし、トレーニングデータセットからは独立したテストデータセットを使って最終的な性能評価を得ることができる。
- 非復元抽出法のため、トレーニングデータセットとテストデータセットのペアが与えられたときに、各サンプル点はその中に１回しか現れない。このため、ホールドアウト法よりもバリアンスの低い性能評価が得られる事が利点である。
- k分割交差検証で標準的に用いられるkの値は10であり、ほとんどの場合は妥当な選択である。
- 比較的小さなトレーニングデータセットを扱っている場合は、分割数を増やすと良いかもしれない。
- kの値を増やすと汎化性能の評価に対するバイアスが低くなる。
- k分割交差検証を改善したものが、層化k分割交差検証である。クラスの比率が均等でないケースでは、評価のバイアスとバリアンスが改善される。
- 層化交差検証では、各サブセットでのクラスの比率が維持される。

In [20]:
import numpy as np
from sklearn.cross_validation import StratifiedKFold #イテレータオブジェクト
#分割元データ、分割数、乱数生成器の状態を指定し、層化k分割交差検証イテレータを表すStratifiedKFoldクラスのインスタンス化
kfold = StratifiedKFold(y=y_train,n_folds=10,random_state=1)

scores = []
#イテレータのインデックスと要素をループ処理
#  データをモデルに適合
#  テストデータの正解率を算出
#  リストに正解率を追加
#  分割の番号、0以上の要素数、正解率を出力
for k, (train,test) in enumerate(kfold):
    pipe_lr.fit(X_train[train],y_train[train])
    score = pipe_lr.score(X_train[test],y_train[test])
    scores.append(score)
    print('Fold : %s, Class dist.: %s, Acc : %.3f') %(k+1,np.bincount(y_train[train]),score)
#正解率の平均と標準偏差を出力
print('CV accuracy : %.3f +/- %.3f') %(np.mean(scores),np.std(scores))

Fold : 1, Class dist.: [256 153], Acc : 0.891
Fold : 2, Class dist.: [256 153], Acc : 0.978
Fold : 3, Class dist.: [256 153], Acc : 0.978
Fold : 4, Class dist.: [256 153], Acc : 0.913
Fold : 5, Class dist.: [256 153], Acc : 0.935
Fold : 6, Class dist.: [257 153], Acc : 0.978
Fold : 7, Class dist.: [257 153], Acc : 0.933
Fold : 8, Class dist.: [257 153], Acc : 0.956
Fold : 9, Class dist.: [257 153], Acc : 0.978
Fold : 10, Class dist.: [257 153], Acc : 0.956
CV accuracy : 0.950 +/- 0.029


In [21]:
#より効率的なプログラミング
from sklearn.cross_validation import cross_val_score
#交差検証のcross_val_score関数でモデルの正解率を算出
#推定器estimator,トレーニングデータX,予測値y,分割数cv,CPU数n_jobsを指定
scores = cross_val_score(estimator=pipe_lr,X=X_train,y=y_train,cv=10,n_jobs=1)
print('CV accuracy scores: %s') %scores
print('CV accuracy: %.3f +/- %.3f') %(np.mean(scores),np.std(scores))

CV accuracy scores: [ 0.89130435  0.97826087  0.97826087  0.91304348  0.93478261  0.97777778
  0.93333333  0.95555556  0.97777778  0.95555556]
CV accuracy: 0.950 +/- 0.029


- cross_val_score関数のアプローチの極めて便利な特徴の１つとして、サブセット毎の評価の計算処理をマシン上の複数のCPUに分散させることができる。